# Setup

In [ ]:
import azureml.core
print(azureml.core.__version__)
import pandas as pd
import numpy as np
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

from pandas.tseries.frequencies import to_offset

from automl.client.core.nativeclient import AutoMLForecaster
from automl.client.core.nativeclient import AutoMLNativeClient
from azureml.core.authentication import AbstractAuthentication

np.set_printoptions(precision=8, suppress=True, linewidth=120)

## Authenticate to the server

In [ ]:
from azureml.core import Workspace

# Create the workspace using the specified parameters
workspace = Workspace.create(name = 'workspace name',
                             subscription_id = 'subscription id',
                             resource_group = 'resource group name', 
                             location = 'region',
                             create_resource_group = True,
                             sku = 'basic',
                             exist_ok = True)

# Data
For the demonstration purposes we will generate the data artificially and use them for the forecasting.

In [ ]:
TIME_COLUMN_NAME = 'date'
GRAIN_COLUMN_NAME = 'grain'
TARGET_COLUMN_NAME = 'y'

def get_timeseries(train_len: int,
                   test_len: int,
                   time_column_name: str,
                   target_column_name: str,
                   grain_column_name: str,
                   grains: int = 1,
                   freq: str = 'H'):
    """
    Return the time series of designed length.

    :param train_len: The length of training data (one series).
    :type train_len: int
    :param test_len: The length of testing data (one series).
    :type test_len: int
    :param time_column_name: The desired name of a time column.
    :type time_column_name: str
    :param
    :param grains: The number of grains.
    :type grains: int
    :param freq: The frequency string representing pandas offset.
                 see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
    :type freq: str
    :returns: the tuple of train and test data sets.
    :rtype: tuple

    """
    data_train = []  # type: List[pd.DataFrame]
    data_test = []  # type: List[pd.DataFrame]
    data_length = train_len + test_len
    for i in range(grains):
        X = pd.DataFrame({
            time_column_name: pd.date_range(start='2000-01-01',
                                            periods=data_length,
                                            freq=freq),
            target_column_name: np.arange(data_length).astype(float),
            'universal_answer': np.repeat(42, data_length),
            grain_column_name: np.repeat('g{}'.format(i), data_length)
        })
        data_train.append(X[:train_len])
        data_test.append(X[train_len:])
    X_train = pd.concat(data_train)
    y_train = X_train.pop(target_column_name).values
    X_test = pd.concat(data_test)
    y_test = X_test.pop(target_column_name).values
    return X_train, y_train, X_test, y_test

n_test_periods = 6
n_train_periods = 12
X_train, y_train, X_test, y_test = get_timeseries(train_len=n_train_periods,
                                                  test_len=n_test_periods,
                                                  time_column_name=TIME_COLUMN_NAME,
                                                  target_column_name=TARGET_COLUMN_NAME,
                                                  grain_column_name=GRAIN_COLUMN_NAME,
                                                  grains=2)
X_train

# Create the configuration and run the forecasting.
First generate the configuration, in which we:
* Set metadata columns: target, time column and grain column names.
* Ask for 10 iterations through models, last of which will represent the Ensemble of previous ones.
* Validate our data using cross validation with rolling window method.
* Set normalized root mean squared error as a metric to select the best model.
* Finally, we set the task to be forecasting.
* By default, we apply the lag lead operator and rolling window to the target value i.e. we use the previous values as a predictor for the future ones.

In [ ]:
time_series_settings = {
    'primary_metric': 'normalized_root_mean_squared_error',
    'iteration_timeout_minutes': 5,
    'iterations': 10,
    'n_cross_validations': 5,
    'time_column_name': TIME_COLUMN_NAME,
    'grain_column_names': GRAIN_COLUMN_NAME,
    'max_horizon': n_test_periods,
    'client': AutoMLNativeClient(workspace=workspace)
}

Run the model selection and training process.

In [ ]:
automl_forecaster = AutoMLForecaster(**time_series_settings)
local_run = automl_forecaster.fit(
    X=X_train,
    y=y_train,
    compute_target='local',
    show_output=True
)
# Retrieve the best model to use it further.
_, fitted_model = local_run.get_output()

# Test the fitted model.

Fitted model has two remarkable methods.
* predict - for the back compatibility with scikit-learn pipeline. It will return all y values for all horizons and hence its output shape will not align row by row with X_test.
* forecast - the method designed specifically for forecasting. It returns the tuple: y values which has the same row number as X_test and the whole transformed data frame. This data frame and y values array will contain data for the most recent origin times.

In [ ]:
y_pred = fitted_model.predict(X_test)
print("Predicted(forecasted) values size: {}".format(y_pred.shape[0]))
print("The number of rows in X_test: {}".format(X_test.shape[0]))
# Note that y_pred cannot be aligned with X_test.

**Optional part** <br/>
Why y_pred is not aligned to the x test?
To answer this question let us dissect fitted_model which is a scikit learn pipeline and make the time series transform.

In [ ]:
# By default, the first transform in the pipeline is a time series transform.
# It generates multiple horizons.
# Each step in the scikit learn pipeline is a tuple of name and transform.
print(fitted_model.steps[0][0]) # Print the name of 1st transform.
# And apply it to X_test
X_test_transformed = fitted_model.steps[0][1].transform(X_test)
print("The number of rows in transformed X_test is: {}".format(X_test_transformed.shape[0]))
# y_pred is aligned with transformed data frame.
X_test_transformed['Forecast'] = y_pred
# For the brevity we will remove all but horizon and forecast.
#X_test_transformed.head()
X_transformed_short = X_test_transformed[['horizon_origin', 'Forecast']]
X_transformed_short.head(6)

X_transform is a "forecast request", requesting 42 numbers. Multiple values, corresponding to different origin times and therefore horizon, are requested for one forecast period. This is necessary for training and evaluation.<br>
Note that now data frame contains the origin time column name. This column is created by the lag lead and rolling window transform. Each new row contain lag and rolling window values corresponding to different horizons. <br>
Each line in the data frame is the forecast for period in date using data up to and including origin.

**End of optional part**

### The forecast() function.

In this section we will review four scenarios.
* X_train is followed by the X_test.
* There is a gap between X_test and X_train.
* There is no gap between X_test and X_train, but y_test is partially known.
* No X_test and X_train, were provided, we need to infer all data up to destination date.
* The "ragged" data: there is context in the y_test, but also there is a gap between X_train and X_test.

We will test the accuracy of the predictions using MAPE.

#### X_train is followed by the X_test.

In [ ]:
y_pred_no_gap, xy_nogap =  fitted_model.forecast(X_test, np.repeat(np.NaN, X_test.shape[0]))
# xy_nogap contains y_pred_no_gap in the _automl_target_col column.
# The data set contains hourly data, the training set ends at 01/01/2000 at  11:00
# the testing data start at 12:00
xy_nogap

After the application of the forecast function only the latest horizins are retained.

In [ ]:
# We will left joun the X_transformed_short with all horizons to the xy_nogap.
# We will trim all predictors and rename the  _automl_target_col and horizon_origin
# to Forecast_func and horizon_origin_func
xy_short = xy_nogap[['horizon_origin', '_automl_target_col']]
xy_short.rename(columns={"horizon_origin": "horizon_origin_func", "_automl_target_col": "Forecast_func"}, inplace=True)
merged = pd.merge(X_transformed_short, xy_short, how='left', left_index=True, right_index=True)
merged.head(6)

Note that ```forecast``` removed all non recent horizons.

#### There is a gap between X_test and X_train.
First lets remove by two data points from X_test and then apply the pipeline.

In [ ]:
dfs = []
for grain, df in X_test.groupby(GRAIN_COLUMN_NAME):
    dfs.append(df[2:])
X_gap = pd.concat(dfs)
del dfs # Make sure we cleaned memory after concatenation is complete.
# If we have a gap in data, we need to set ignore_data_errors to true,
# because otherwise the gap in data will be treated as error.
# In this case testing set starts at 14:00
y_pred_gap, xy_gap =  fitted_model.forecast(X_gap, np.repeat(np.NaN, X_gap.shape[0]), ignore_data_errors=True)
xy_gap

Note, that the result was provided beginning from 14:00.

#### There is no gap between X_test and X_train, but y_test is partially known.
In this example we will generate supply by two y_preds to each grains.

In [ ]:
X_test[TARGET_COLUMN_NAME] = y_test
dfs = []
for grain, df in X_test.groupby(GRAIN_COLUMN_NAME):
    # We need to group our data frame by grain, and add y back to it
    # because we need to make sure that y is aligned to the corresponding X.
    y = df[TARGET_COLUMN_NAME]
    y[2:] = np.NaN
    df[TARGET_COLUMN_NAME] = y
    dfs.append(df)
X_y = pd.concat(dfs)
y_part = X_y.pop(TARGET_COLUMN_NAME).values
del dfs # Make sure we cleaned memory after concatenation is complete.
y_pred_known, xy_known =  fitted_model.forecast(X_y, y_part)
xy_known

Here, the y passed in was returned back, and NaNs filled in.

#### No X_test and X_train, were provided, we need to infer all data up to destination date.

In [ ]:
#For the fair comparison of the results we will take the destination date as a last date in the test set.
dest = max(X_test[TIME_COLUMN_NAME])
y_pred_dest, xy_dest = fitted_model.forecast(forecast_destination=dest)
xy_dest

#### The "ragged" data: there is context in the y_test, but also there is a gap between X_train and X_test.

In [ ]:
# We will shift X_test by two hours
dfs = []
for grain, df in X_test.groupby(GRAIN_COLUMN_NAME):
    # We need to group our data frame by grain, and add y back to it
    # because we need to make sure that y is aligned to the corresponding X.
    y = [14., 15., np.NaN, np.NaN, np.NaN, np.NaN]
    df[TARGET_COLUMN_NAME] = y
    # Shift dates
    start = min(df[TIME_COLUMN_NAME]) + 2 * to_offset('H')
    df[TIME_COLUMN_NAME] = pd.date_range(start = start, freq='H', periods=df.shape[0])
    dfs.append(df)
X_ragged = pd.concat(dfs)
y_ragged = X_ragged.pop(TARGET_COLUMN_NAME).values
y_pred_ragged, xy_ragged = fitted_model.forecast(X_ragged, y_ragged)
xy_ragged

Note that the forecast starts at 16:00.

### Test the results using MAPE.

In [ ]:
# Define the MAPE function  for the benchmarking our results.
def MAPE(actual, pred):
    actual_ft = actual.astype(float)
    not_na = ~(np.isnan(actual_ft))
    not_zero = ~np.isclose(actual_ft, 0.0)
    actual_safe = actual_ft[not_na & not_zero]
    pred_safe = pred[not_na & not_zero]
    APE = 100*np.abs((actual_safe - pred_safe)/actual_safe)
    return np.mean(APE)

We need to trim xy_nogap, xy_dest and X_test to make comparisons.

In [ ]:
def trim_df(X):
    dfs = []
    for grain, df in X.groupby(GRAIN_COLUMN_NAME):
        dfs.append(df[2:])
    return pd.concat(dfs)
# X_test already contains y.
X_test_trimmed = trim_df(X_test)
xy_nogap_trimmed = trim_df(xy_nogap)
xy_dest_trimmed = trim_df(xy_dest)

Before we will be able to estimate MAPE we need to make sure that all values are aligned. We will set index to grain and date and sort data frame with it.

In [ ]:
X_test_trimmed.set_index([TIME_COLUMN_NAME, GRAIN_COLUMN_NAME], inplace=True)
X_test_trimmed.sort_index(inplace=True)
xy_nogap_trimmed.sort_index(inplace=True)
xy_dest_trimmed.sort_index(inplace=True)
xy_gap.sort_index(inplace=True)
xy_known.sort_index(inplace=True)

Finally estimate MAPEs of all forecasts. We need to mention that in the transformed data set returned by forecast() method the column with forecast is called _automl_target_column.

In [ ]:
# Do the actual comparisons of MAPES.
print("MAPE of data set with no gap: {}".format(MAPE(X_test_trimmed[TARGET_COLUMN_NAME].values,
                                                     xy_nogap_trimmed['_automl_target_col'].values)))
print("MAPE of data set with gap: {}".format(MAPE(X_test_trimmed[TARGET_COLUMN_NAME].values,
                                                  xy_gap['_automl_target_col'].values)))
print("MAPE of data set with previous y knowlege: {}".format(MAPE(X_test_trimmed[TARGET_COLUMN_NAME].values,
                                                                  xy_known['_automl_target_col'].values)))
print("MAPE of forecast with target date: {}".format(MAPE(X_test_trimmed[TARGET_COLUMN_NAME].values,
                                                          xy_dest_trimmed['_automl_target_col'].values)))
print("MAPE of forecast with ragged data: {}".format(MAPE(np.repeat(np.array([16., 17., 18., 19.]), 2),
                                                          xy_ragged['_automl_target_col'].values)))